In [1]:
import os
import numpy as np
import pandas as pd
import tools as tl
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def loadData(files, subset, coralSuccess=70, lagColumnNames=[], shiftValues=[]):
    df = tl.loadRuns(files, subset)
    df = tl.addPercent(df) ## turn node type counts into percents
    df = tl.scaledByNumberOfNodes(df)
    df = tl.labelCrashStatistics(df, coralSuccess)
    df = tl.normalize(df)
    df = tl.addLag(df, lagColumnNames, shiftValues)
    return(df)

## Overview of model runs

In [2]:
topDirectory = "./output/" #os.getcwd()
files, overviewOfRuns = tl.viewRuns(topDirectory)

overviewOfRuns

,Rows,Columns,Grid Option,Grazing,Initial Coral Percent,Initial Macroalgae Percent,r,d,a,y,Time,Record Rate,Number of Simulations
0,15,15,0,60,33,33,10,40,20,75,5010,200,100
1,15,15,0,50,33,33,10,40,20,75,5010,500,100
2,15,15,0,50,33,33,10,40,20,75,5010,200,100
3,15,15,0,40,33,33,10,40,20,75,5010,200,100
4,15,15,0,70,33,33,10,40,20,75,5010,200,100
5,15,15,0,55,33,33,10,40,20,75,5010,500,100
6,15,15,0,30,33,33,10,40,20,75,5010,200,100
7,15,15,2,50,33,33,10,40,20,75,5010,500,100
8,15,15,2,55,33,33,10,40,20,75,5010,500,100
9,20,20,0,50,33,33,10,40,20,75,5010,500,100


## Choose Runs to Explore Via Index

In [3]:
filesOfInterest = [1,7,5,8]  ## e.g. [2,3,6] -> will pull simulations with corresponding index in table above

subset = overviewOfRuns.loc[filesOfInterest,:]
df = loadData(files, subset, coralSuccess=60)
df

## OR Choose Run to Explore Via Model Parameters

In [5]:
subset = overviewOfRuns[(overviewOfRuns['Grazing'] > 25) &
                        (overviewOfRuns['Grazing'] < 75) &
                        (overviewOfRuns['Grid Option'] == 0) &
                        (overviewOfRuns['Time'] == 5010)]
df = loadData(files, subset, coralSuccess=60)

,Simulation,Timestep,CoralCount,TurfCount,MacroalgaeCount,Coral-CoralNeighbors,Turf-TurfNeighbors,Macro-MacroNeighbors,CoralPatchCount,AlgaePatchCount,TurfPatchCount,MacroPatchCount,File
0,0,0,67,85,73,0.29,0.43,0.31,12,1,15,14,1
1,0,500,2,87,136,0.12,0.47,0.66,1,1,6,1,1
2,0,1000,0,75,150,0.00,0.38,0.67,1,1,8,2,1
3,0,1500,0,82,143,0.00,0.45,0.69,1,1,10,2,1
4,0,2000,0,71,154,0.00,0.36,0.71,1,1,16,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,99,3000,123,77,25,0.64,0.37,0.35,3,4,6,5,8
1096,99,3500,169,46,10,0.77,0.20,0.44,1,19,22,3,8
1097,99,4000,169,56,0,0.77,0.29,0.00,1,15,15,1,8
1098,99,4500,182,43,0,0.84,0.29,0.00,1,13,13,1,8


## Grazing Temporal Plots

In [ ]:
subset = overviewOfRuns[(overviewOfRuns['Grazing'] > 25) &
                        (overviewOfRuns['Grazing'] < 75) &
                        (overviewOfRuns['Grid Option'] == 0) &
                        (overviewOfRuns['Time'] == 5010)]
df = loadData(files, subset, coralSuccess=60)

### Boxplot

In [ ]:
coralData.groupby(['File']).boxplot(column=['CoralCount'], by=['Timestep'],figsize=(15,5))

### Seaborn

In [ ]:
sns.relplot(x="Timestep", y="CoralPercent", row='Grazing', hue='coralSuccess',
            size='Simulation', data=df, kind="line", height=2, aspect=3, alpha=.6)

## Spatial Metrics Histograms  - Neighbors

In [ ]:
subset = overviewOfRuns[(overviewOfRuns['Grazing'] ==50) &
                        (overviewOfRuns['Grid Option'] == 0) &
                        (overviewOfRuns['Time'] == 5010) &
                        (overviewOfRuns['Initial Coral Percent'] == 50)]
df = loadData(files, subset, coralSuccess=60)
df.columns()

In [ ]:
fig, ax = plt.subplots(len(variables), len(timesteps), figsize=(7,6), sharey='row', sharex=True)

variables = ['coral_neighbors','macroalgae_neighbors','turf_neighbors']
timesteps = [0,200,800]
colors = {-1:'#EAD3CD', 0:'#956285', 1:'#2A203C'}
legend_keys = {-1:'algae dominant', 0:'metastable', 1:'coral dominant'}
ylabels = ['Coral', ' Macroalgae', 'Turf' ]
row = 0

for variable in variables:
    for column in range(0,len(timesteps)):
        for success in [-1,1,0]:
            array = df[(df['Timestep']==timesteps[column]) & (df['coralSuccess'] == success)][variable]
            hist, bins = np.histogram(array, bins=15)
            ax[row,column].bar(bins[:-1], hist.astype(np.float32) / hist.sum(), width=(bins[1]-bins[0]), color=colors[success], alpha=.8, label=legend_keys[success])        
        ax[0,column].set_title('Timestep = ' + str(timesteps[column]),fontsize=10)        
    ax[row,0].set_ylabel(ylabels[row] + 'Patch Count')
    row = row + 1
    
ax[0,0].legend(loc='upper left')
#fig.suptitle('Normalized average same-species neighbors for simulations with ' + '\n' +'varying final coral successes :: Grazing=.5, Initial Coral=50%', fontsize=17) # or plt.suptitle('Main title')

fig.tight_layout()
fig.subplots_adjust(top=.9, wspace=.1, hspace=.2)
neighbors_plot = plt
#plt.savefig('figures_paper/hist_neighbors_small.png', dpi=200)
plt.show()